In [7]:
import torch
import numpy as np
import pandas as pd
import cv2
import os
from sklearn.utils import shuffle

ModuleNotFoundError: No module named 'sklearn'

In [2]:
if torch.cuda.is_available():
    print("GPU is available.")
else:
    print("GPU is not available.")

GPU is available.


In [3]:
def load_images(image_path, target_size=(100,100)):
    img = cv2.imread(image_path)
    img = cv2.resize(img, target_size)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img / 255.0

In [5]:
cats = 'data/train/cat/'
dogs = 'data/train/dog/'

cat_imgs = [load_images(cats + file) for file in os.listdir(cats) if file.endswith('.jpg')]
dog_imgs = [load_images(dogs + file) for file in os.listdir(dogs) if file.endswith('.jpg')]

cat_labels = [0] * len(cat_imgs)
dog_labels = [1] * len(dog_imgs)

In [8]:
x = np.array(cat_imgs + dog_imgs)
y = np.array(cat_labels + dog_labels)

x, y = shuffle(x, y)

print("Image data shape: ", x.shape())
print("Labels Shape: ". y.shape())

NameError: name 'cat_labels' is not defined